In [2]:
import torch
import torchvision
import einops
from torch import nn
from IPython.display import display

pil=torchvision.transforms.ToPILImage()#将tensor数据转换成图片

加载手写数据集,minist数据集是灰度图像，并且图片的size都是（28， 28， 1）

In [ ]:

mnist=torchvision.datasets.MNIST("dataset",download=True)
print(mnist.data.shape)#torch.Size([60000, 28, 28])
image=mnist.data[0]#第一张图片
display(pil(image))#显示第一张图片
display(image.shape)#显示图片的大小

下面将进行图片的切片,(pw w)代表pw * w= 28即原始维度，即将图片横切2次，竖切2次
最后，将一张图片分成了4个14*14的小图片

In [25]:
result=einops.rearrange(
    image.reshape((28,28)),"(pw w) (ph h) -> (pw ph) w h",pw=2,ph=2
)
for i in range(4):
    display(pil(result[i]))
del mnist

In [20]:
from einops import rearrange
a=torch.randn((1,28,28))
#下面这种方法将一张图片分为16个小块，每一个小块的长宽被展平成为了一维的向量49个像素点
rearrange(a,"b (pw w) (ph h) -> b (pw ph) (w h)",pw=4,ph=4).shape
#下面需要手动repeat
a=torch.Tensor(((1,1),(2,2),(3,3)))
a=a.reshape(1,3,2)
einops.repeat(a,"() h w -> repeat h w",repeat=3) #将a重复三份

torch.Size([3, 3, 2])

上面的测试结束，下面开始正式进行构建vision transformer的网络结构

In [5]:
from einops.layers.torch import Rearrange
from torch import nn
import torch
import einops
from torchvision import transforms

class VisionTransformer(nn.Module):
    def __init__(self):
        super().__init__()
        #然后开始定义该网络涉及到的一些需要用的函数
        self.patch_embedding=nn.Sequential(
            Rearrange("b (pw w) (ph h) -> b (pw ph) (w h)",pw=4,ph=4),#将得到的每个小图片的长乘以宽的像素展平，得到16*49的向量
            nn.Linear(7*7,7*7), #每张小图片的向量为49，输出也为49
        )
        # shape: (batch,1, 7*7)
        #通过这样的方式就能将改变量加入到该模型的变量中，然后才能自动进行求导
        self.class_token=nn.parameter.Parameter(torch.randn(1,1,7*7))
        # shape: (batch, 4*4+1, 7*7)
        self.position_encodings=nn.parameter.Parameter(
            torch.randn((1,4*4+1,7*7))
        )
        #使用python自带的transfoermerEncoder结构
        #d_model是embedding向量的维数，设置它为7*7=49
        #nhead是MSA中self attention的个数，设置nhead=7
        #dim_feedforward是隐藏层的隐藏节点个数
        #btach_size是将张量的第一维指定为batch
        #3:the number of sub-encoder-layers in the encoder (required).
        self.transformer_encoder=nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=7*7,nhead=7,dim_feedforward=128,batch_first=True
            ),
            3,
        )
        #最后进行分类的MLP层，由于有十个类，因此需要将最后输出结果变为10个
        #nn.LayerNorm是对向量进行归一化，7*7表示对行进行归一化
        self.mlp_head=nn.Sequential(nn.LayerNorm(7*7),nn.Linear(7*7,10))

    #下面将定义该网络的前向传播过程
    def forward(self,x):
        x=self.patch_embedding(x)#首先将该批图片进行embedding，即剪裁和展平、线性变换
        batch_size, _ , _=x.shape#将X的第一维数赋值给batch_size，即一次输入了多少张图片
        #给每张图片(batch_size张图片)矩阵在最后都上class_token，得到class_token
        class_token=einops.repeat(
            self.class_token,
            "() words features -> repeat words features",
            repeat=batch_size
        )
        x = torch.cat((class_token,x),dim=-2)#即在第负二维（行）上进行拼接，然后就变成了17行
        x += self.position_encodings #然后再将x加入position_encoding，但是它可以通过训练得到
        x=self.transformer_encoder(x)#接下来将加了class_token和position_embeeding的最终向量输入encode
        x=x[:,0]#只取class_token那部分进行预测，class_token在第一行
        x=self.mlp_head(x)#利用class_token取进行预测
        return x#返回结果


下面开始实例化一个模型，然后下载数据集，分为训练集和测试集

In [10]:
vision_transformer_model=VisionTransformer()#实例化一个模型
#Compose组合几个变换方法，并且按照顺序对数据进行相应变换
#即先把数据转换成tensor，然后再按照平均值、标准差做归一化
transform=transforms.Compose(
    [transforms.ToTensor(),transforms.Normalize((0.1307,),(0.3081))]
)
train_set=torchvision.datasets.MNIST("dataset",train=True,download=False,transform=transform)
test_set=torchvision.datasets.MNIST("dataset",train=False,transform=transform)

开始构建训练代码

In [15]:
def train(model,optimizer,scheduler,criterion,epochs,train_loader):
    model.train()
    for epoch in range(epochs):
        for batch_idx,(x,y) in enumerate(train_loader):
            x=einops.rearrange(x,"b c h w->(b c) h w")  #将X变成3维数据
            optimizer.zero_grad()#梯度归零，避免叠加
            output=model(x)#得到输出
            loss=criterion(output,y)#计算输出与标签之间的损失
            loss.backward()#反向传播，计算梯度
            optimizer.step()#根据梯度进行参数更新

            if batch_idx%10==0:
                print(f"epoch:{epoch},i:{batch_idx*len(x)}/{len(train_loader.dataset)},loss:{loss.item()}")
        scheduler.step()#更新学习率
        

下面开始指定训练的参数，包括损失函数，优化器等

In [16]:
epochs=4
batch_size=64
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(vision_transformer_model.parameters(),lr=0.001)#adam优化器
scheduler=torch.optim.lr_scheduler.StepLR(optimizer,step_size=1,gamma=0.5)#对该优化器学习率进行调整，每一个epoch之后，将该学习率乘以0.5，即减半
train_loader=torch.utils.data.DataLoader(train_set,batch_size=batch_size)
#DataLoader()函数的作用是将train_set随机打乱，并将其分为一个个batch，每个batch有多少个数据

开始训练，并进行模型保存

In [17]:
train(vision_transformer_model,optimizer,scheduler,criterion,epochs,train_loader)
torch.save(vision_transformer_model,"E:\\desktop\\王艳\\我的\\14周\\models\\result.pt")

epoch:0,i:0/60000,loss:2.8362410068511963
epoch:0,i:640/60000,loss:2.272383689880371
epoch:0,i:1280/60000,loss:2.189990520477295
epoch:0,i:1920/60000,loss:1.8391051292419434
epoch:0,i:2560/60000,loss:1.4220757484436035
epoch:0,i:3200/60000,loss:1.164360523223877
epoch:0,i:3840/60000,loss:0.9684287309646606
epoch:0,i:4480/60000,loss:0.9003849625587463
epoch:0,i:5120/60000,loss:1.1698224544525146
epoch:0,i:5760/60000,loss:0.7308517694473267
epoch:0,i:6400/60000,loss:0.6272963881492615
epoch:0,i:7040/60000,loss:0.45971372723579407
epoch:0,i:7680/60000,loss:0.4639527201652527
epoch:0,i:8320/60000,loss:0.5405558943748474
epoch:0,i:8960/60000,loss:0.39258474111557007
epoch:0,i:9600/60000,loss:0.5786784887313843
epoch:0,i:10240/60000,loss:0.8854106664657593
epoch:0,i:10880/60000,loss:0.3221619129180908
epoch:0,i:11520/60000,loss:0.8119263648986816
epoch:0,i:12160/60000,loss:0.47405335307121277
epoch:0,i:12800/60000,loss:0.42560896277427673
epoch:0,i:13440/60000,loss:0.4382953345775604
epoch:0

下面利用测试集进行精度测试

In [32]:
test_loader=torch.utils.data.DataLoader(test_set,batch_size=1)
correct=0#正确的个数
correct_rate=0#正确率
for idx,(x,y) in enumerate(test_loader):
    x=einops.rearrange(x,"b c h w->(b c) h w")
    output=vision_transformer_model(x)
    _,max_result=output.max(axis=1)
    if max_result.equal(y):
        correct+=1
    correct_rate=correct/(idx+1)
    print(f"在{idx+1}个测试中，有{correct}个预测正确，正确率为{100*correct_rate}%")


在1个测试中，有1个预测正确，正确率为100.0%
在2个测试中，有2个预测正确，正确率为100.0%
在3个测试中，有3个预测正确，正确率为100.0%
在4个测试中，有4个预测正确，正确率为100.0%
在5个测试中，有5个预测正确，正确率为100.0%
在6个测试中，有6个预测正确，正确率为100.0%
在7个测试中，有7个预测正确，正确率为100.0%
在8个测试中，有8个预测正确，正确率为100.0%
在9个测试中，有8个预测正确，正确率为88.88888888888889%
在10个测试中，有9个预测正确，正确率为90.0%
在11个测试中，有10个预测正确，正确率为90.9090909090909%
在12个测试中，有11个预测正确，正确率为91.66666666666666%
在13个测试中，有12个预测正确，正确率为92.3076923076923%
在14个测试中，有13个预测正确，正确率为92.85714285714286%
在15个测试中，有14个预测正确，正确率为93.33333333333333%
在16个测试中，有15个预测正确，正确率为93.75%
在17个测试中，有16个预测正确，正确率为94.11764705882352%
在18个测试中，有17个预测正确，正确率为94.44444444444444%
在19个测试中，有18个预测正确，正确率为94.73684210526315%
在20个测试中，有19个预测正确，正确率为95.0%
在21个测试中，有20个预测正确，正确率为95.23809523809523%
在22个测试中，有21个预测正确，正确率为95.45454545454545%
在23个测试中，有22个预测正确，正确率为95.65217391304348%
在24个测试中，有23个预测正确，正确率为95.83333333333334%
在25个测试中，有24个预测正确，正确率为96.0%
在26个测试中，有25个预测正确，正确率为96.15384615384616%
在27个测试中，有26个预测正确，正确率为96.29629629629629%
在28个测试中，有27个预测正确，正确率为96.42857142857143%
在29个测试中，有28个预测正确，正确率为96.55172413793103%
在30个测试中，